# 第８章 デザインとリファクタリング

## 8.2　コードのデザイン
### 8.2.4　結合度 

In [7]:
# 密結合している2つの関数の例
import pandas as pd
from scipy.stats import linregress


def process_sdg_data(input_excel_file, columns_to_drop):
    df = pd.read_excel(input_excel_file)
    df = df.drop(columns_to_drop, axis=1)
    df = df.set_index("GeoAreaName").transpose()
    return df


def fit_trendline(country_name):
    df = process_sdg_data(
        "../data/SG_GEN_PARL.xlsx",
        [
            "Goal",
            "Target",
            "Indicator",
            "SeriesCode",
            "SeriesDescription",
            "GeoAreaCode",
            "Reporting Type",
            "Sex",
            "Units",
        ],
    )
    timestamps = [int(i) for i in df.index.tolist()]
    data = df[country_name].tolist()

    result = linregress(timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared

In [8]:
fit_trendline("India")

(np.float64(0.292), np.float64(0.869))

In [9]:
# 密結合でない（疎結合な）例

import pandas as pd
from scipy.stats import linregress


def fit_trendline(year_timestamps, data):
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared


def process_sdg_data(input_excel_file, columns_to_drop):
    df = pd.read_excel(input_excel_file)
    df = df.drop(columns_to_drop, axis=1)
    df = df.set_index("GeoAreaName").transpose()
    return df


df = process_sdg_data(
    "../data/SG_GEN_PARL.xlsx",
    [
        "Goal",
        "Target",
        "Indicator",
        "SeriesCode",
        "SeriesDescription",
        "GeoAreaCode",
        "Reporting Type",
        "Sex",
        "Units",
    ],
)
timestamps = [int(i) for i in df.index.tolist()]
country_data = df["India"].tolist()
slope, r_squared = fit_trendline(timestamps, country_data)

In [12]:
slope

np.float64(0.292)

## 8.3　ノートブックからスケーラブルなスクリプトへ
### 8.3.2　ノートブックからスクリプトの作成

In [13]:
import pandas as pd

In [14]:
def process_sdg_data(input_excel_file, columns_to_drop, output_csv_file):
    df = pd.read_excel(input_excel_file)
    df = df.drop(columns_to_drop, axis=1)
    df = df.set_index("GeoAreaName").transpose()
    df.to_csv(output_csv_file, index="GeoAreaName")

In [16]:
import os


def test_process_sdg_data():

    test_filepath = "test_sgd_data.csv"

    process_sdg_data(
        "SG_GEN_PARL.xlsx",
        [
            "Goal",
            "Target",
            "Indicator",
            "SeriesCode",
            "SeriesDescription",
            "GeoAreaCode",
            "Reporting Type",
            "Sex",
            "Units",
        ],
        test_filepath,
    )

    df = pd.read_csv(test_filepath)

    assert len(df) == 24
    assert len(df.columns) == 196

    # cleanup step - delete the file produced in the test
    os.remove(test_filepath)

In [17]:
test_process_sdg_data()